In [ ]:
! pip install optuna
import optuna
import pandas as pd
import numpy as np

# R Statistical test
import joblib
! pip install rpy2
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

utils = importr('utils')
utils.install_packages("forecast")
forecast_r = importr("forecast")


In [ ]:
hyper = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/hyperparametertuning_study.pkl")

In [ ]:
#! pip install tikzplotlib
#! pip install SciencePlots
#! sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended cm-super

import tikzplotlib
from matplotlib import pyplot as plt
plt.style.reload_library()
plt.style.use(['science'])
plt.plot([1,2],[2,5])
plt.title("Some Title her")
plt.savefig('fig1.pdf')


In [ ]:
def model(x, p):
    return x ** (2 * p + 1) / (1 + x ** (2 * p))


pparam = dict(xlabel='Voltage (mV)', ylabel='Current ($\mu$A)')

x = np.linspace(0.75, 1.25, 201)

with plt.style.context(['science']):
    fig, ax = plt.subplots()
    for p in [10, 15, 20, 30, 50, 100]:
        ax.plot(x, model(x, p), label=p)
    ax.legend(title='Order')
    ax.autoscale(tight=True)
    ax.set(**pparam)

In [ ]:
hyper.best_params

In [ ]:
optuna.visualization.plot_optimization_history(hyper)

In [ ]:
hyper.best_trial

In [ ]:
optuna.visualization.plot_param_importances(hyper)

In [ ]:
optuna.visualization.plot_slice(hyper)

# Baseline - Alle industrier - Generel model

## Q1

In [ ]:
import joblib
Q1_performance = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_performance_base.pkl")
Q1_test = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_base.pkl")
Q1_train = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_train_base.pkl")

Q1_test["error_anal"] = Q1_test.EPS_actual - Q1_test.analyst_EPS_mean
Q1_test["error_model"] = Q1_test.EPS_actual - Q1_test.Model_predicted


In [ ]:
d = Q1_test["error_anal"] - Q1_test["error_model"]


In [ ]:
import joblib
! pip install rpy2
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

Q1_performance = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_performance_base.pkl")
Q1_test = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_base.pkl")
Q1_train = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_train_base.pkl")

Q1_test["error_anal"] = Q1_test.analyst_EPS_mean - Q1_test.EPS_actual 
Q1_test["error_model"] =Q1_test.Model_predicted - Q1_test.EPS_actual

utils = importr('utils')
utils.install_packages("forecast")
forecast_r = importr("forecast")



In [ ]:
Q1_performance

In [ ]:
with localconverter(ro.default_converter + pandas2ri.converter):
  Q1_r = ro.conversion.py2rpy(Q1_test)

q1_p = forecast_r.dm_test(Q1_r.rx2["error_anal"],Q1_r.rx2["error_model"],alternative="greater",power=1).rx2["p.value"]

## Q2

In [ ]:
Q2_performance = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_performance_base.pkl")
Q2_test = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_test_base.pkl")
Q2_train = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_train_base.pkl")

Q2_test["error_anal"] = Q2_test.analyst_EPS_mean - Q2_test.EPS_actual 
Q2_test["error_model"] =Q2_test.Model_predicted - Q2_test.EPS_actual

with localconverter(ro.default_converter + pandas2ri.converter):
  Q2_r = ro.conversion.py2rpy(Q2_test)

q2_p = forecast_r.dm_test(Q2_r.rx2["error_anal"],Q2_r.rx2["error_model"],alternative="greater",power=1).rx2["p.value"]



## Q4

In [ ]:
Q4_performance = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_performance_base.pkl")
Q4_test = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_base.pkl")
Q4_train = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_train_base.pkl")

Q4_test["error_anal"] = Q4_test.analyst_EPS_mean - Q4_test.EPS_actual 
Q4_test["error_model"] =Q4_test.Model_predicted - Q4_test.EPS_actual

with localconverter(ro.default_converter + pandas2ri.converter):
  Q4_r = ro.conversion.py2rpy(Q4_test)

q4_p = forecast_r.dm_test(Q4_r.rx2["error_anal"],Q4_r.rx2["error_model"],alternative="greater",power=1).rx2["p.value"]

In [ ]:
Q4_test[Q4_test.GVKEY==1690]

In [ ]:
samlet_df_pværdier= pd.DataFrame(data=[q1_p,q2_p,q4_p]).T.rename(columns={0: "Q1",1: "Q2",2: "Q4"})
samlet_df_pværdier

joblib.dump(samlet_df_pværdier,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/samlet_pværdier_base.pkl")

# Industri Model vs subset

In [ ]:
def p_value (generel_df,industri_df, industri):
  generel_df[industri]["model_error"] =generel_df[industri]["EPS_actual"]- generel_df[industri]["Model_predicted"]
  industri_df[industri]["model_error"] =industri_df[industri]["EPS_actual"]- industri_df[industri]["Model_predicted"]

  with localconverter(ro.default_converter + pandas2ri.converter):
    temp_gen = ro.conversion.py2rpy(generel_df[industri])
  with localconverter(ro.default_converter + pandas2ri.converter):
    temp_ind = ro.conversion.py2rpy(industri_df[industri])

  p_value_ = forecast_r.dm_test(temp_ind.rx2["model_error"],temp_gen.rx2["model_error"],alternative="greater",power=1).rx2["p.value"]
  p_value_ = np.asarray(p_value_,dtype=np.float32)

  return p_value_


## Q1

In [ ]:
Q1_test_generel = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_generel.pkl")
Q1_test_industri = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_industri.pkl")
industri_list = []
for k, v in Q1_test_industri.items():
  industri_list.append(k)
print(industri_list)


In [ ]:
p_values_arr = []
for industri in industri_list:
  p_values_arr.append(p_value(Q1_test_generel,Q1_test_industri,industri=industri))

p_values_arr = np.asarray(p_values_arr,dtype=np.float32).flatten()

p_value_df = pd.DataFrame(data=[industri_list,p_values_arr]).T.rename(columns={0:"Industri",1:"P_værdi"})
p_value_df["P_værdi"]=p_value_df.P_værdi.astype("float32")
p_value_df.round(4)
p_value_df
joblib.dump(p_value_df,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/pvalue_industri_Q1")

## Q2

In [ ]:
Q2_test_generel = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_test_generel.pkl")
Q2_test_industri = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_test_industri.pkl")


In [ ]:
p_values_arr = []
for industri in industri_list:
  p_values_arr.append(p_value(Q2_test_generel,Q2_test_industri,industri=industri))

p_values_arr = np.asarray(p_values_arr,dtype=np.float32).flatten()

p_value_df = pd.DataFrame(data=[industri_list,p_values_arr]).T.rename(columns={0:"Industri",1:"P_værdi"})
p_value_df["P_værdi"]=p_value_df.P_værdi.astype("float32")
p_value_df.round(4)
p_value_df
joblib.dump(p_value_df,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/pvalue_industri_Q2")

## Q4

In [ ]:
Q4_test_generel = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_generel.pkl")
Q4_test_industri = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_industri.pkl")


In [ ]:
p_values_arr = []
for industri in industri_list:
  p_values_arr.append(p_value(Q4_test_generel,Q4_test_industri,industri=industri))

p_values_arr = np.asarray(p_values_arr,dtype=np.float32).flatten()

p_value_df = pd.DataFrame(data=[industri_list,p_values_arr]).T.rename(columns={0:"Industri",1:"P_værdi"})
p_value_df["P_værdi"]=p_value_df.P_værdi.astype("float32")
p_value_df.round(4)
joblib.dump(p_value_df,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/pvalue_industri_Q4")

In [ ]:
p_value_df.info()

# Market CAP

In [ ]:
def p_value_market (generel_df,cap):
  generel_df[cap]["model_error"] =generel_df[cap]["Model_predicted"]- generel_df[cap]["EPS_actual"]
  generel_df[cap]["analyst_error"] =generel_df[cap]["analyst_EPS_mean"]- generel_df[cap]["EPS_actual"]

  with localconverter(ro.default_converter + pandas2ri.converter):
    r_temp = ro.conversion.py2rpy(generel_df[cap])

  p_value_ = forecast_r.dm_test(r_temp.rx2["analyst_error"],r_temp.rx2["model_error"],alternative="greater",power=1).rx2["p.value"]
  p_value_ = np.asarray(p_value_,dtype=np.float32)

  return p_value_

In [ ]:
Q1_market = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_marketcap.pkl")
Q2_market = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_test_marketcap.pkl")
Q4_market = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_marketcap.pkl")

cap_liste = ["small_cap","mid_cap","large_cap"]
Q_files = [Q1_market,Q2_market,Q4_market]

In [ ]:
p_values_arr = []
for quarters in Q_files:

  for cap in cap_liste:
    p_values_arr.append(p_value_market(quarters,cap))

  
p_values_arr = np.asarray(p_values_arr,dtype=np.float32).flatten()

market_cap_pval = pd.DataFrame({"small_cap":p_values_arr[np.r_[0,3,6]], "mid_cap":p_values_arr[np.r_[1,4,7]],"large_cap":p_values_arr[np.r_[2,5,8]]}).round(5)
market_cap_pval.set_index(pd.Series(["1Q","2Q","4Q"]),inplace=True)
market_cap_pval

In [ ]:
# Eksporter til en joblib fil

joblib.dump(market_cap_pval,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/marketcap_pvalues_samlet.pkl")

# Industri subset vs analytikere

In [ ]:
def p_value_industri (generel_df,industri):
  generel_df[industri]["model_error"] =generel_df[industri]["Model_predicted"]- generel_df[industri]["EPS_actual"]
  generel_df[industri]["analyst_error"] =generel_df[industri]["analyst_EPS_mean"]- generel_df[industri]["EPS_actual"]

  with localconverter(ro.default_converter + pandas2ri.converter):
    r_temp = ro.conversion.py2rpy(generel_df[industri])

  p_value_ = forecast_r.dm_test(r_temp.rx2["analyst_error"],r_temp.rx2["model_error"],alternative="greater",power=1).rx2["p.value"]
  p_value_ = np.asarray(p_value_,dtype=np.float32)

  return p_value_

In [ ]:
Q1_industri = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_generel.pkl")
Q2_industri = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_test_generel.pkl")
Q4_industri = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_generel.pkl")

industri_liste = []
for k,v in Q1_industri.items():
  industri_liste.append(k)

Q_files = [Q1_industri,Q2_industri,Q4_industri]

In [ ]:
p_values_arr = []
for quarters in Q_files:

  for industri in industri_liste:
    p_values_arr.append(p_value_industri(quarters,industri))

  
p_values_arr = np.asarray(p_values_arr,dtype=np.float32).flatten()


In [ ]:
p_values_array_split = np.split(p_values_arr,3)

industri_generel_samlet = pd.DataFrame(np.transpose(p_values_array_split),columns=["Q1","Q2","Q4"]).round(3).set_index(pd.Series(industri_liste))

In [ ]:
joblib.dump(industri_generel_samlet,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/industri_generel_samlet_pværdier.pkl")
industri_generel_samlet

In [ ]:
industri_generel_samlet < 0.05


In [ ]:
Q1_perf_gen = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_performance_generel.pkl")
Q2_perf_gen = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_performance_generel.pkl")
Q4_perf_gen = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_performance_generel.pkl")

Q1_perf_gen = pd.concat(Q1_perf_gen)
Q2_perf_gen = pd.concat(Q2_perf_gen)
Q4_perf_gen = pd.concat(Q4_perf_gen)

In [ ]:
df_1 = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_generel.pkl")

In [ ]:
df_1["Energy"]["EPS_actual"].describe()

In [ ]:
df_1["Business Equipment"]["EPS_actual"].mean()

In [ ]:
for x in [Q1_perf_gen,Q2_perf_gen,Q4_perf_gen]:
  x.drop("MASE",axis=1,inplace=True)

In [ ]:
Q1_perf_gen.reset_index(level=1).to_excel("test.xlsx")

In [ ]:
pd.concat(joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_performance_generel.pkl"))

# LORT

In [ ]:
#

In [ ]:
test_medium = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_marketcap.pkl") ["mid_cap"]

In [ ]:
test_medium = test_medium[test_medium.industry_fama!="Healthcare"]

In [ ]:
test_medium["model_error"] =abs(test_medium["Model_predicted"]- test_medium["EPS_actual"])
test_medium["analyst_error"] =abs(test_medium["analyst_EPS_mean"]- test_medium["EPS_actual"])

In [ ]:
test_medium[["model_error","analyst_error"]].max()

In [ ]:
Q1_m = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_performance_marketcap.pkl")
Q2_m = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_performance_marketcap.pkl")
Q4_m = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_performance_marketcap.pkl")


In [ ]:
pd.concat(Q1_m)

In [ ]:
pd.concat(Q2_m)

In [ ]:
pd.concat(Q4_m)

# Test af outliers

In [ ]:
Q1_outliers = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_base.pkl")

In [ ]:
Q1_outliers["percentage"] = abs(Q1_outliers["EPS_actual"] / Q1_outliers["analyst_EPS_mean"] -1)

In [ ]:
Q1_outliers[Q1_outliers.percentage<1]

# Coverage og Spredning

## Coverage

In [ ]:
def p_value_covspred (generel_df,label,quarter):
  generel_df[label]["model_error"] =generel_df[label]["Model_predicted"]- generel_df[label]["EPS_actual"]
  generel_df[label]["analyst_error"] =generel_df[label]["analyst_EPS_mean"]- generel_df[label]["EPS_actual"]

  with localconverter(ro.default_converter + pandas2ri.converter):
    r_temp = ro.conversion.py2rpy(generel_df[label])
  if quarter ==1:
    p_value_ = forecast_r.dm_test(r_temp.rx2["analyst_error"],r_temp.rx2["model_error"],alternative="two.sided",power=1).rx2["p.value"]
  else:
    p_value_ = forecast_r.dm_test(r_temp.rx2["analyst_error"],r_temp.rx2["model_error"],alternative="greater",power=1).rx2["p.value"]
  
  p_value_ = np.asarray(p_value_,dtype=np.float32)
  return p_value_

In [ ]:
Q1_coverage = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_coverage.pkl")
Q2_coverage = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_test_coverage.pkl")
Q4_coverage = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_coverage.pkl")

label_list = ["low","mediumlow","mediumhigh","high"]
Q_files = [Q1_coverage,Q2_coverage,Q4_coverage]
names = ["Q1","Q2","Q4"]

In [ ]:
p_values_arr = []
for quarters in Q_files:
  if quarters["high"].index[-1].month == 6:
    for label in label_list:
      p_values_arr.append(p_value_covspred(quarters,label,1))

  else:
    for label in label_list:
      p_values_arr.append(p_value_covspred(quarters,label,2))

  
p_values_arr = np.asarray(p_values_arr,dtype=np.float32).flatten()

coverage_pval = pd.DataFrame({"low":p_values_arr[np.r_[0,4,8]], "medlow":p_values_arr[np.r_[1,5,9]],"medhigh":p_values_arr[np.r_[2,6,10]],"high":p_values_arr[np.r_[3,7,11]]})
coverage_pval.set_index(pd.Series(["1Q","2Q","4Q"]),inplace=True)
coverage_pval


In [ ]:
coverage_pval.round(3).T

In [ ]:
joblib.dump(coverage_pval,"/content/drive/MyDrive/Cand.fælles/Speciale/Python/coverage_pval.pkl")

## Spredning

In [ ]:
Q1_dispersion = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q1_test_dispersion.pkl")
Q2_dispersion = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q2_test_dispersion.pkl")
Q4_dispersion = joblib.load("/content/drive/MyDrive/Cand.fælles/Speciale/Python/Q4_test_dispersion.pkl")

label_list = ["low","mediumlow","mediumhigh","high"]
Q_files = [Q1_dispersion,Q2_dispersion,Q4_dispersion]
names = ["Q1","Q2","Q4"]

In [ ]:
p_values_arr = []
for quarters in Q_files:
  if quarters["high"].index[-1].month == 6:
    for label in label_list:
      p_values_arr.append(p_value_covspred(quarters,label,1))

  else:
    for label in label_list:
      p_values_arr.append(p_value_covspred(quarters,label,2))

  
p_values_arr = np.asarray(p_values_arr,dtype=np.float32).flatten()

dispersion_pval = pd.DataFrame({"low":p_values_arr[np.r_[0,4,8]], "medlow":p_values_arr[np.r_[1,5,9]],"medhigh":p_values_arr[np.r_[2,6,10]],"high":p_values_arr[np.r_[3,7,11]]})
dispersion_pval.set_index(pd.Series(["1Q","2Q","4Q"]),inplace=True)
dispersion_pval


In [ ]:
(dispersion_pval).T.round(3)